#### Feature Engineering ####
The process of `feature engineering` includes following steps:

- Brainstorming or Testing features;
- Deciding what features to create;
- Creating features;
- Checking how the features work with your model;
- Improving your features if needed;
- Go back to brainstorming/creating more features until the work is done.

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
import holidays as hd
import calendar
from datetime import datetime, date
from pprint import pprint

In [2]:
cycle_usage = pd.read_csv("cycleusage_cleansed.csv")
cycle_usage.head()

,StartStation Id,Start Date,EndStation Id,End Date,Duration,StartStation Id Used,EndStation Id Used,Frequency,StartStation Address,StartStation latitude,...,EndStation Address,EndStation latitude,EndStation longitude,EndStation capacity,distance (geodesic),Daily Weather,Hourly Weather,Humidity,Windspeed,Apparent Temperature (Avg)
0,836,06/09/2018 18:03,749,06/09/2018 18:38,2100,325,49668,1,"York Way, Kings Cross",51.541596,...,"Haggerston Road, Haggerston",51.539329,-0.074285,37,3.558160,partly-cloudy-day,"[{'time': 1536188400, 'summary': 'Clear', 'ico...",0.72,6.26,NaN
1,831,24/07/2018 20:17,638,24/07/2018 20:39,1320,2294,68917,5,"St John's Crescent, Brixton",51.466232,...,"Falcon Road, Clapham Junction",51.466634,-0.169821,28,3.936338,partly-cloudy-night,"[{'time': 1532386800, 'summary': 'Clear', 'ico...",0.62,6.89,NaN
2,831,06/06/2018 09:33,264,06/06/2018 09:56,1380,2294,74294,1,"St John's Crescent, Brixton",51.466232,...,"Tysoe Street, Clerkenwell",51.526443,-0.109257,20,6.704526,partly-cloudy-day,"[{'time': 1528239600, 'summary': 'Partly Cloud...",0.70,8.11,NaN
3,831,18/04/2018 08:44,372,18/04/2018 09:12,1680,2294,50130,2,"St John's Crescent, Brixton",51.466232,...,"Sardinia Street, Holborn",51.515208,-0.117863,24,5.458654,clear-day,"[{'time': 1524006000, 'summary': 'Partly Cloud...",0.65,3.12,NaN
4,831,15/08/2018 08:45,41,15/08/2018 09:14,1740,2294,88345,1,"St John's Crescent, Brixton",51.466232,...,"Pindar Street, Liverpool Street",51.520955,-0.083494,24,6.427970,partly-cloudy-day,"[{'time': 1534287600, 'summary': 'Partly Cloud...",0.71,10.70,NaN


In [3]:
rm_columns = {
    #"StartStation Id",
    #"Start Date",
    "StartStation Address",
   # "StartStation capacity",
    #"EndStation Id",
    "End Date",
    "EndStation Address",
   # "EndStation capacity",
   # "Duration",
   # "Frequency",
    "Humidity",
    "Windspeed",
    "Apparent Temperature (Avg)",
    "StartStation Id Used",
    "EndStation Id Used",
    "StartStation latitude",
    "StartStation longitude",
    "EndStation latitude",
    "EndStation longitude",
    "Hourly Weather",
   # "distance (geodesic)"
   # "Daily Weather"
}

cycle_usage.drop(columns=rm_columns, inplace=True)
cycle_usage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 470190 entries, 0 to 470189
Data columns (total 9 columns):
StartStation Id          470190 non-null int64
Start Date               470190 non-null object
EndStation Id            470190 non-null int64
Duration                 470190 non-null int64
Frequency                470190 non-null int64
StartStation capacity    470190 non-null int64
EndStation capacity      470190 non-null int64
distance (geodesic)      470190 non-null float64
Daily Weather            470190 non-null object
dtypes: float64(1), int64(6), object(2)
memory usage: 32.3+ MB


In [4]:
# Check for empty values and empty strings
np.where(pd.isnull(cycle_usage))
np.where(cycle_usage.applymap(lambda x: x == ''))

(array([], dtype=int64), array([], dtype=int64))

#### Darksky note:#####
> Our system is presently very simple: it finds the “worst” weather condition that will happen during the day (4AM to 4AM), and uses the icon for it. The only case where a daily icon will show a *-night value is partly-cloudy-night, and this is done to match the daily summary text. We already have intentions to change this behavior, because it is confusing. 
In the meantime, you can assume that if partly-cloudy-night is the worst weather condition that was found, that it was clear during the day. So you can just treat partly-cloudy-night as an alias for clear-day. 

In [5]:
print(cycle_usage.iloc[452861]["Start Date"][:10], cycle_usage.iloc[452861]["Start Date"][-5:])

12/12/2017 08:45


In [6]:
cycle_usage.groupby(by="Daily Weather").count()
cycle_usage["Daily Weather"].loc[cycle_usage["Daily Weather"]=="partly-cloudy-night"] = "clear-day"

D:\Anaconda3\envs\prana\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
# Inconsistent dates e.g. 00:00 01:25:00
cycle_usage.iloc[452861]
for index, p in cycle_usage.iterrows():
    if (len(p["Start Date"]) == 19):
      #  print(index, p["Start Date"])
        cycle_usage["Start Date"].iloc[index] = p["Start Date"][:16]
    elif (len(p["Start Date"]) > 19):
        print("anomaly", index, p["Start Date"])
        cycle_usage["Start Date"].iloc[index] = p["Start Date"][:10] + " " +p["Start Date"][-5:]

cycle_usage.dropna(inplace=True)
cycle_usage.count()


anomaly 452861 12/12/2017 2017 08:45


StartStation Id          470190
Start Date               470190
EndStation Id            470190
Duration                 470190
Frequency                470190
StartStation capacity    470190
EndStation capacity      470190
distance (geodesic)      470190
Daily Weather            470190
dtype: int64

Adding weekdays (Monday, Tuesday...)

In [8]:
#Add weekdays
cycle_usage["Start Date"] =  pd.to_datetime(cycle_usage["Start Date"], format='%d/%m/%Y %H:%M')
cycle_usage['Weekday'] = cycle_usage.apply(lambda row: calendar.day_name[row["Start Date"].weekday()],axis=1)

In [9]:
cycle_usage.head()

,StartStation Id,Start Date,EndStation Id,Duration,Frequency,StartStation capacity,EndStation capacity,distance (geodesic),Daily Weather,Weekday
0,836,2018-09-06 18:03:00,749,2100,1,24,37,3.558160,partly-cloudy-day,Thursday
1,831,2018-07-24 20:17:00,638,1320,5,25,28,3.936338,clear-day,Tuesday
2,831,2018-06-06 09:33:00,264,1380,1,25,20,6.704526,partly-cloudy-day,Wednesday
3,831,2018-04-18 08:44:00,372,1680,2,25,24,5.458654,clear-day,Wednesday
4,831,2018-08-15 08:45:00,41,1740,1,25,24,6.427970,partly-cloudy-day,Wednesday



`Meteorologische Jahreszeiten` <br>
Nördliche Hemisphäre <br>
Frühling: 1. März bis 31. Mai <br>
Sommer: 1. Juni bis 31. August <br>
Herbst: 1. September bis 30. November <br>
Winter: 1. Dezember bis 28. Februar <br>

In [10]:
#Add seasons
def seasons(p):
    """Get meteorological season"""
    year = int(str(p["Start Date"])[:4])
    date_m = p["Start Date"]
    if date_m >= datetime(year, 3, 1, 0,0,0) and date_m <= datetime(year, 5, 31, 23,59,59):
        return "Spring"
    elif date_m >= datetime(year, 6, 1, 0,0,0) and date_m <= datetime(year, 8, 31, 23,59,59):
        return "Summer"
    elif date_m >= datetime(year, 9, 1, 0,0,0) and date_m <= datetime(year, 11, 30, 23,59,59):
        return "Autumn"
    elif date_m >= datetime(year, 12, 1, 0,0,0) or date_m < datetime(year, 3, 1, 23,59,59):
        return "Winter"
        
cycle_usage['Season'] = cycle_usage.apply(lambda row: seasons(row),axis=1)
cycle_usage.head()

,StartStation Id,Start Date,EndStation Id,Duration,Frequency,StartStation capacity,EndStation capacity,distance (geodesic),Daily Weather,Weekday,Season
0,836,2018-09-06 18:03:00,749,2100,1,24,37,3.558160,partly-cloudy-day,Thursday,Autumn
1,831,2018-07-24 20:17:00,638,1320,5,25,28,3.936338,clear-day,Tuesday,Summer
2,831,2018-06-06 09:33:00,264,1380,1,25,20,6.704526,partly-cloudy-day,Wednesday,Summer
3,831,2018-04-18 08:44:00,372,1680,2,25,24,5.458654,clear-day,Wednesday,Spring
4,831,2018-08-15 08:45:00,41,1740,1,25,24,6.427970,partly-cloudy-day,Wednesday,Summer


 ##### Day & Night cycle ####
 After 6 pm night, after 6 am day → “Day” & “Night”

In [11]:
#Day night state
def daynight(p):
    """Returns day or night depending on clock"""
    timeclock = p["Start Date"].hour
    if timeclock > 6 and timeclock < 18:
        return "day"
    else:
        return "night"       
cycle_usage["Day & Night"] = cycle_usage.apply(lambda row: daynight(row), axis=1)

###### Holidays ######
Check if that day is a specific holiday?

In [12]:
#Consider holidays (e.g. Good Friday in UK)
def holiday(p):
    """ Checks if holiday """
    uk_holidays = hd.UK()
    if (p["Start Date"].date() in uk_holidays):
        return True
    else:
        return False
    
for date2, name in sorted(hd.UK(state='London', years=[2015,2016,2017], observed=False).items()):
    print(date2, name)
    
cycle_usage["Holiday"] = cycle_usage.apply(lambda row: holiday(row), axis=1)            
cycle_usage.head()

2015-01-01 New Year's Day
2015-01-02 New Year Holiday [Scotland]
2015-03-17 St. Patrick's Day [Northern Ireland]
2015-04-03 Good Friday
2015-04-06 Easter Monday [England, Wales, Northern Ireland]
2015-05-04 May Day
2015-05-25 Spring Bank Holiday
2015-07-12 Battle of the Boyne [Northern Ireland]
2015-08-03 Summer Bank Holiday [Scotland]
2015-08-31 Late Summer Bank Holiday [England, Wales, Northern Ireland]
2015-11-30 St. Andrew's Day [Scotland]
2015-12-25 Christmas Day
2015-12-26 Boxing Day
2016-01-01 New Year's Day
2016-01-02 New Year Holiday [Scotland]
2016-03-17 St. Patrick's Day [Northern Ireland]
2016-03-25 Good Friday
2016-03-28 Easter Monday [England, Wales, Northern Ireland]
2016-05-02 May Day
2016-05-30 Spring Bank Holiday
2016-07-12 Battle of the Boyne [Northern Ireland]
2016-08-01 Summer Bank Holiday [Scotland]
2016-08-29 Late Summer Bank Holiday [England, Wales, Northern Ireland]
2016-11-30 St. Andrew's Day [Scotland]
2016-12-25 Christmas Day
2016-12-26 Boxing Day
2017-01-01

,StartStation Id,Start Date,EndStation Id,Duration,Frequency,StartStation capacity,EndStation capacity,distance (geodesic),Daily Weather,Weekday,Season,Day & Night,Holiday
0,836,2018-09-06 18:03:00,749,2100,1,24,37,3.558160,partly-cloudy-day,Thursday,Autumn,night,False
1,831,2018-07-24 20:17:00,638,1320,5,25,28,3.936338,clear-day,Tuesday,Summer,night,False
2,831,2018-06-06 09:33:00,264,1380,1,25,20,6.704526,partly-cloudy-day,Wednesday,Summer,day,False
3,831,2018-04-18 08:44:00,372,1680,2,25,24,5.458654,clear-day,Wednesday,Spring,day,False
4,831,2018-08-15 08:45:00,41,1740,1,25,24,6.427970,partly-cloudy-day,Wednesday,Summer,day,False


In [13]:
# Months
def months_names(p):
    """Returns month name"""
    months = {
        1: "January",
        2: "February",
        3: "March",
        4: "April",
        5: "May",
        6: "June",
        7: "July",
        8: "August",
        9: "September",
        10: "October",
        11: "November",
        12: "December"
    }
    return months.get(p["Start Date"].month, "not defined")

cycle_usage["Month"] = cycle_usage.apply(lambda row: months_names(row), axis=1)

##### Split Start Date #####
> Dates are difficult to handle for ML. Idea: splitting to several columns

In [14]:
#Extract only dd-mm-YYYY
cycle_usage['Date'] = cycle_usage.apply(lambda row: str(row["Start Date"])[:10], axis=1)
cycle_usage['Date'] = pd.to_datetime(cycle_usage.Date, format="%Y/%m/%d")
#Extracting Year
cycle_usage['Year'] = cycle_usage['Date'].dt.year
#Extracting Month
cycle_usage['Month'] = cycle_usage['Date'].dt.month
#Extracting passed years since the date
cycle_usage['Passed_Years'] = date.today().year - cycle_usage['Date'].dt.year
#Extracting passed months since the date
cycle_usage['Passed_Months'] = (date.today().year - cycle_usage['Date'].dt.year) * 12 + date.today().month - cycle_usage['Date'].dt.month
cycle_usage.head()

,StartStation Id,Start Date,EndStation Id,Duration,Frequency,StartStation capacity,EndStation capacity,distance (geodesic),Daily Weather,Weekday,Season,Day & Night,Holiday,Month,Date,Year,Passed_Years,Passed_Months
0,836,2018-09-06 18:03:00,749,2100,1,24,37,3.558160,partly-cloudy-day,Thursday,Autumn,night,False,9,2018-09-06,2018,1,7
1,831,2018-07-24 20:17:00,638,1320,5,25,28,3.936338,clear-day,Tuesday,Summer,night,False,7,2018-07-24,2018,1,9
2,831,2018-06-06 09:33:00,264,1380,1,25,20,6.704526,partly-cloudy-day,Wednesday,Summer,day,False,6,2018-06-06,2018,1,10
3,831,2018-04-18 08:44:00,372,1680,2,25,24,5.458654,clear-day,Wednesday,Spring,day,False,4,2018-04-18,2018,1,12
4,831,2018-08-15 08:45:00,41,1740,1,25,24,6.427970,partly-cloudy-day,Wednesday,Summer,day,False,8,2018-08-15,2018,1,8


In [15]:
cycle_usage.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 470190 entries, 0 to 470189
Data columns (total 18 columns):
StartStation Id          470190 non-null int64
Start Date               470190 non-null datetime64[ns]
EndStation Id            470190 non-null int64
Duration                 470190 non-null int64
Frequency                470190 non-null int64
StartStation capacity    470190 non-null int64
EndStation capacity      470190 non-null int64
distance (geodesic)      470190 non-null float64
Daily Weather            470190 non-null object
Weekday                  470190 non-null object
Season                   470190 non-null object
Day & Night              470190 non-null object
Holiday                  470190 non-null bool
Month                    470190 non-null int64
Date                     470190 non-null datetime64[ns]
Year                     470190 non-null int64
Passed_Years             470190 non-null int64
Passed_Months            470190 non-null int64
dtypes: bool(1), dat

In [16]:
rm_columns = {
    #"StartStation Id",
    "Start Date",
    "StartStation Address",
    #"StartStation capacity",
    #"EndStation Id",
    "End Date",
    "EndStation Address",
    "EndStation capacity",
    "Duration",
   # "Frequency",
    "Humidity",
    "Windspeed",
    "Apparent Temperature (Avg)",
    "StartStation Id Used",
    "EndStation Id Used",
    "StartStation latitude",
    "StartStation longitude",
    "EndStation latitude",
    "EndStation longitude",
    "Hourly Weather",
    "distance (geodesic)"
   # "Daily Weather"
}

cycle_usage.drop(columns=rm_columns, inplace=True, errors="ignore")
cycle_usage.drop_duplicates(inplace=True)
cycle_usage.head()

,StartStation Id,EndStation Id,Frequency,StartStation capacity,Daily Weather,Weekday,Season,Day & Night,Holiday,Month,Date,Year,Passed_Years,Passed_Months
0,836,749,1,24,partly-cloudy-day,Thursday,Autumn,night,False,9,2018-09-06,2018,1,7
1,831,638,5,25,clear-day,Tuesday,Summer,night,False,7,2018-07-24,2018,1,9
2,831,264,1,25,partly-cloudy-day,Wednesday,Summer,day,False,6,2018-06-06,2018,1,10
3,831,372,2,25,clear-day,Wednesday,Spring,day,False,4,2018-04-18,2018,1,12
4,831,41,1,25,partly-cloudy-day,Wednesday,Summer,day,False,8,2018-08-15,2018,1,8


In [17]:
cycle_usage.count()

StartStation Id          470190
EndStation Id            470190
Frequency                470190
StartStation capacity    470190
Daily Weather            470190
Weekday                  470190
Season                   470190
Day & Night              470190
Holiday                  470190
Month                    470190
Date                     470190
Year                     470190
Passed_Years             470190
Passed_Months            470190
dtype: int64

In [18]:
cycle_usage.iloc[67]

StartStation Id                          744
EndStation Id                            171
Frequency                                  7
StartStation capacity                     30
Daily Weather                           rain
Weekday                              Tuesday
Season                                Winter
Day & Night                              day
Holiday                                False
Month                                     12
Date                     2015-12-15 00:00:00
Year                                    2015
Passed_Years                               4
Passed_Months                             40
Name: 67, dtype: object

In [19]:
#Get the most frequently occurring record
cycle_usage.groupby(by="StartStation Id").count().sort_values("Frequency", ascending=False).head(1)

,EndStation Id,Frequency,StartStation capacity,Daily Weather,Weekday,Season,Day & Night,Holiday,Month,Date,Year,Passed_Years,Passed_Months
StartStation Id,,,,,,,,,,,,,
109,776,776,776,776,776,776,776,776,776,776,776,776,776


In [20]:
cycle_usage = cycle_usage[cycle_usage["StartStation Id"] == 109]

In [21]:
cycle_usage.to_csv("features.csv", header=True, index=False)